__Fake News Classifier Using Bidirectional LSTM__

In [3]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd

In [7]:
df = pd.read_csv('train.csv')

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
df.shape

(20800, 5)

In [11]:
df = df.dropna()

In [12]:
df.shape

(18285, 5)

In [14]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
# get the independent features
x = df.drop('label',axis=1)

In [16]:
y = df['label']

In [17]:
# check where dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [18]:
import tensorflow as tf

In [19]:
tf.__version__

'2.13.0'

In [20]:
from tensorflow import keras
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import Dense

In [21]:
# vocabulary_size
voc_size = 5000

In [22]:
messages = x.copy()

In [23]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [24]:
import re
import nltk
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emon1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
messages.reset_index(inplace=True)

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [29]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[1947, 2562, 2584, 952, 3080, 824, 3236, 1734, 4090, 1396], [810, 4886, 139, 1737, 812, 2573, 2167], [1407, 3353, 1773, 4723], [3827, 133, 202, 4469, 1135, 2], [1969, 812, 1657, 1462, 2253, 3323, 812, 961, 1752, 3113], [3492, 115, 4959, 4334, 4639, 3618, 1583, 2354, 1778, 423, 4386, 2844, 2821, 4083, 2167], [4318, 2091, 2962, 1735, 4578, 3203, 711, 3165, 2389, 987, 1682], [343, 4343, 842, 4564, 378, 2705, 3618, 4648, 2389, 987, 1682], [1473, 4523, 3279, 1316, 2436, 77, 3492, 3858, 3618, 4979], [3108, 4375, 1547, 2868, 4663, 2660, 2889, 76], [2882, 4826, 2103, 2214, 4611, 4099, 3466, 4092, 361, 4880, 37], [4469, 1266, 3080, 77, 3618, 378], [1334, 2411, 4430, 3576, 810, 908, 4535, 886, 490], [4631, 3195, 1757, 3187, 4337, 1737, 2564, 2389, 987, 1682], [4494, 4776, 888, 3345, 4660, 2389, 987, 1682], [3885, 4769, 1255, 3077, 2779, 4494, 3683, 1415, 4227, 4658], [4437, 1298, 4886], [465, 3762, 492, 2560, 3618, 1545, 586, 2167], [4638, 4394, 139, 2012, 1386, 3788, 598, 2963, 3323], [2702, 2

In [30]:
onehot_repr

[[1947, 2562, 2584, 952, 3080, 824, 3236, 1734, 4090, 1396],
 [810, 4886, 139, 1737, 812, 2573, 2167],
 [1407, 3353, 1773, 4723],
 [3827, 133, 202, 4469, 1135, 2],
 [1969, 812, 1657, 1462, 2253, 3323, 812, 961, 1752, 3113],
 [3492,
  115,
  4959,
  4334,
  4639,
  3618,
  1583,
  2354,
  1778,
  423,
  4386,
  2844,
  2821,
  4083,
  2167],
 [4318, 2091, 2962, 1735, 4578, 3203, 711, 3165, 2389, 987, 1682],
 [343, 4343, 842, 4564, 378, 2705, 3618, 4648, 2389, 987, 1682],
 [1473, 4523, 3279, 1316, 2436, 77, 3492, 3858, 3618, 4979],
 [3108, 4375, 1547, 2868, 4663, 2660, 2889, 76],
 [2882, 4826, 2103, 2214, 4611, 4099, 3466, 4092, 361, 4880, 37],
 [4469, 1266, 3080, 77, 3618, 378],
 [1334, 2411, 4430, 3576, 810, 908, 4535, 886, 490],
 [4631, 3195, 1757, 3187, 4337, 1737, 2564, 2389, 987, 1682],
 [4494, 4776, 888, 3345, 4660, 2389, 987, 1682],
 [3885, 4769, 1255, 3077, 2779, 4494, 3683, 1415, 4227, 4658],
 [4437, 1298, 4886],
 [465, 3762, 492, 2560, 3618, 1545, 586, 2167],
 [4638, 4394, 139

__Embedding Representation__

In [31]:
sen_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sen_length)
embedded_docs

array([[   0,    0,    0, ..., 1734, 4090, 1396],
       [   0,    0,    0, ...,  812, 2573, 2167],
       [   0,    0,    0, ..., 3353, 1773, 4723],
       ...,
       [   0,    0,    0, ..., 2389,  987, 1682],
       [   0,    0,    0, ..., 1261, 4576, 1828],
       [   0,    0,    0, ...,  506, 3673, 1451]])

In [32]:
# creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sen_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
import numpy as np

x_final = np.array(embedded_docs)
y_final = np.array(y)

In [34]:
# train test split
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [35]:
# model training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 24s 43ms/step - loss: 0.2632 - accuracy: 0.8841 - val_loss: 0.1922 - val_accuracy: 0.9196
Epoch 2/10
383/383 [==============================] - 14s 36ms/step - loss: 0.1272 - accuracy: 0.9506 - val_loss: 0.2115 - val_accuracy: 0.9155
Epoch 3/10
383/383 [==============================] - 14s 38ms/step - loss: 0.0818 - accuracy: 0.9705 - val_loss: 0.2386 - val_accuracy: 0.9105
Epoch 4/10
383/383 [==============================] - 15s 39ms/step - loss: 0.0481 - accuracy: 0.9829 - val_loss: 0.3081 - val_accuracy: 0.9127
Epoch 5/10
383/383 [==============================] - 14s 36ms/step - loss: 0.0359 - accuracy: 0.9880 - val_loss: 0.3470 - val_accuracy: 0.9044
Epoch 6/10
383/383 [==============================] - 13s 35ms/step - loss: 0.0218 - accuracy: 0.9932 - val_loss: 0.4171 - val_accuracy: 0.9075
Epoch 7/10
383/383 [==============================] - 14s 37ms/step - loss: 0.0125 - accuracy: 0.9967 - val_loss: 0.4620 - val_accuracy:

In [36]:
y_pred = model.predict(x_test)

189/189 [==============================] - 3s 10ms/step


In [37]:
y_pred

array([[9.9982399e-01],
       [1.4262293e-04],
       [1.8334613e-04],
       ...,
       [4.7528674e-04],
       [9.9627692e-01],
       [9.9996763e-01]], dtype=float32)

In [41]:
y_pred = np.where(y_pred >= 0.5,1,0)

In [42]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [43]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3115,  304],
       [ 265, 2351]], dtype=int64)

In [44]:
print(accuracy_score(y_test,y_pred))

0.9057166528583265
